In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data_folder = r"Downloads\ML/"

In [ ]:
abalone = pd.read_csv(data_folder+"abalone/abalone.csv")
abalone.describe()

In [ ]:
abalone.columns

In [ ]:
abalone = abalone.apply(lambda x: x**(1/3) if x.name in ["Length","Diameter","Whole_weight","Shucked_weight","Viscera_weight","Shell_weight"] else x)
abalone.describe()

In [ ]:
abalone = pd.get_dummies(abalone,columns=["Sex"])
#abalone.head()
#abalone.loc[:,'Volume'] = abalone['Length'].values*abalone['Diameter'].values*abalone['Height'].values

In [ ]:
x_abalone = abalone.drop(["Rings"],axis=1)#"Length","Diameter","Height",
y_abalone = abalone.loc[:,"Rings"]

In [ ]:
'''from sklearn.decomposition import PCA
pca = PCA(n_components=5)
pca.fit(x_abalone)
x_abalone = pd.DataFrame(pca.transform(x_abalone))'''

In [ ]:
#highly imbalanced dataset
#CountStatus = pd.value_counts(y_abalone, sort=True)
#print(CountStatus)
#CountStatus.plot.barh()

In [ ]:
x_abalone.hist(bins=30,xlabelsize=0)#skewed data

In [ ]:
#x_abalone.boxplot(rot=90)#no outliers

In [ ]:
'''sm = pd.plotting.scatter_matrix(abalone, alpha=0.2, figsize=(6, 6), diagonal='kde')

#Change label rotation
[s.xaxis.label.set_rotation(45) for s in sm.reshape(-1)]
[s.yaxis.label.set_rotation(0) for s in sm.reshape(-1)]

#May need to offset label when rotating to prevent overlap of figure
[s.get_yaxis().set_label_coords(-0.3,0.5) for s in sm.reshape(-1)]

#Hide all ticks
[s.set_xticks(()) for s in sm.reshape(-1)]
[s.set_yticks(()) for s in sm.reshape(-1)]

plt.show()'''

In [ ]:
'''f = plt.figure(figsize=(19, 15))
plt.matshow(abalone.corr(), fignum=f.number)
plt.xticks(range(abalone.select_dtypes(['number']).shape[1]), abalone.select_dtypes(['number']).columns, fontsize=14, rotation=45)
plt.yticks(range(abalone.select_dtypes(['number']).shape[1]), abalone.select_dtypes(['number']).columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16)'''

In [ ]:
abalone.corr() #correlation matrix
#abalone.cov() #covarience matrix

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from imblearn.over_sampling import RandomOverSampler,SMOTE,SMOTENC,ADASYN,BorderlineSMOTE,KMeansSMOTE,SVMSMOTE

sm = SMOTE(random_state = 2)
abalone_subset = abalone.groupby('Rings').filter(lambda x : len(x)>5)
abalone_subset = pd.get_dummies(abalone_subset,columns=["Sex"])
x_abalone_sub = abalone_subset.drop(["Length","Diameter","Height","Rings"],axis=1)
y_abalone_sub = abalone_subset.loc[:,"Rings"]
x_abalone_bal, y_abalone_bal = sm.fit_resample(x_abalone_sub, y_abalone_sub.ravel())

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_abalone_bal = scaler.fit_transform(x_abalone_bal)

In [ ]:
#try doing pca to find any meaningful features and remove redundancy
#pca = PCA(n_components=5)
#pca.fit(x_abalone_bal)
#x_abalone_bal = pd.DataFrame(pca.transform(x_abalone_bal))
#x_abalone_bal.head()
#var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

#print(var1)

In [ ]:
#import matplotlib.pyplot as plt
#plot principle components to visualize any chance of seperation among the classes
#plt.scatter(x_abalone_bal[0],x_abalone_bal[1],c=y_abalone_bal)

In [ ]:
CountStatus = pd.value_counts(y_abalone_bal, sort=True)
#print(CountStatus)
CountStatus.plot.barh()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_abalone, y_abalone, test_size = 0.33, random_state = 0)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct

def regression(X_train,Y_train,X_test,Y_test):
    
    lr = LinearRegression()
    lr.fit(X_train,Y_train)
    y_train_lr_pred = np.rint(lr.predict(X_train))
    y_test_lr_pred = np.rint(lr.predict(X_test))
    print("Train and Test accuracy with LR: ",accuracy_score(y_train_lr_pred,Y_train),accuracy_score(y_test_lr_pred,Y_test))
    
    svr = SVR(kernel = 'rbf')
    svr.fit(X_train, Y_train)
    y_train_svr_pred = np.rint(svr.predict(X_train))
    y_test_svr_pred = np.rint(svr.predict(X_test))
    print("Train and Test accuracy with SVR: ",accuracy_score(y_train_svr_pred,Y_train),accuracy_score(y_test_svr_pred,Y_test))
    
    kernel = DotProduct()# + WhiteKernel()
    #kernel = 1.0 * RBF(1.0)
    gpr = GaussianProcessRegressor(kernel=kernel,
             random_state=0).fit(X_train, Y_train)
    y_train_gpr_pred = np.rint(gpr.predict(X_train))
    y_test_gpr_pred = np.rint(gpr.predict(X_test))
    print("Train and Test accuracy with GPR: ",accuracy_score(y_train_gpr_pred,Y_train),accuracy_score(y_test_gpr_pred,Y_test))

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

def classify(X_train,Y_train,X_test,Y_test):
    
    clf_LDA = LinearDiscriminantAnalysis()
    clf_LDA.fit(X_train, Y_train) 
    print("Train and Test accuracy with LDA: ", clf_LDA.score(X_train,Y_train),clf_LDA.score(X_test,Y_test))
    
    clf_ANN = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1,max_iter=2000)
    clf_ANN.fit(X_train, Y_train)
    print("Train and Test accuracy with ANN: ", clf_ANN.score(X_train,Y_train),clf_ANN.score(X_test,Y_test))
    
    clf_KNN = KNeighborsClassifier(n_neighbors=3)
    clf_KNN.fit(X_train, Y_train) 
    print("Train and Test accuracy with KNN k=3: ", clf_KNN.score(X_train, Y_train),clf_KNN.score(X_test,Y_test))
    
    clf_SVC = LinearSVC(max_iter=2000)
    clf_SVC.fit(X_train, Y_train)
    print("Train and Test accuracy with SVC: ", clf_SVC.score(X_train, Y_train),clf_SVC.score(X_test,Y_test))
    
    clf_CNB = ComplementNB()
    clf_CNB.fit(X_train, Y_train)
    print("Train and Test accuracy with Complement Naive Bayes: ", clf_CNB.score(X_train, Y_train),clf_CNB.score(X_test,Y_test))
    
    clf_DT = DecisionTreeClassifier(random_state=0,max_depth=3)
    clf_DT.fit(X_train, Y_train)
    print("Train and Test accuracy with Decision Tree Classifier: ", clf_DT.score(X_train, Y_train),clf_DT.score(X_test,Y_test))
    
    clf_RF = RandomForestClassifier(n_estimators=10random_state=0)
    clf_RF.fit(X_train, Y_train)
    print("Train and Test accuracy with Random Forest Classifier: ", clf_RF.score(X_train, Y_train),clf_RF.score(X_test,Y_test))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

def ensemble(X_train,Y_train,X_test,Y_test):
    
    clf_ADA = AdaBoostClassifier()
    clf_ADA.fit(X_train, Y_train)
    print("Train and Test accuracy with AdaBoost Classifier: ", clf_ADA.score(X_train, Y_train),clf_ADA.score(X_test,Y_test))
    
    clf_GBC = GradientBoostingClassifier()
    clf_GBC.fit(X_train, Y_train)
    print("Train and Test accuracy with GradientBoosting Classifier: ", clf_GBC.score(X_train, Y_train),clf_GBC.score(X_test,Y_test))
    
    clf_HGBC = HistGradientBoostingClassifier()
    clf_HGBC.fit(X_train, Y_train)
    print("Train and Test accuracy with HistGradientBoosting Classifier: ", clf_HGBC.score(X_train, Y_train),clf_HGBC.score(X_test,Y_test))
    
    clf1 = RandomForestClassifier(random_state=0)
    clf2 = KNeighborsClassifier(n_neighbors=2)
    clf3 = SVC(kernel='rbf', probability=True)
    eclf = VotingClassifier(estimators=[('rf', clf1), ('knn', clf2), ('svc', clf3)],
                            voting='soft', weights=[2, 2, 1])
    eclf.fit(X_train, Y_train)
    print("Train and Test accuracy with Voting Classifier(RF,2NN,SVC): ", eclf.score(X_train, Y_train),eclf.score(X_test,Y_test))
    
    estimators = [
    ('knn', KNeighborsClassifier(n_neighbors=2)),
     ('rf', RandomForestClassifier(random_state=0))
      ] #using only 2 of best models so far
    clf_SC = StackingClassifier(
        estimators=estimators, final_estimator=LogisticRegression() )
    clf_SC.fit(X_train, Y_train)
    acc_SC = clf_SC.score(X_test,Y_test)
    print("Train and Test accuracy with Stacking Classifier(2NN,RF): ", clf_SC.score(X_train, Y_train),clf_SC.score(X_test,Y_test))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
regression(x_train,y_train,x_test,y_test)

In [ ]:
classify(x_train,y_train,x_test,y_test)

In [ ]:
ensemble(x_train,y_train,x_test,y_test)